In [ ]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2
import open3d as o3d
from matplotlib import pyplot as plt
from matplotlib import cm

#0605
import zmq
import os
# Directory for data export
output_dir = 'sensor_data'
os.makedirs(output_dir, exist_ok=True)
# Initialize ZeroMQ context and socket for sending data
context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind("tcp://*:5556")

client = carla.Client('localhost', 2000) 
world = client.get_world()
#client.reload_world()

bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [ ]:
# Get the blueprint for the vehicle you want
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 

# Try spawning the vehicle at a randomly chosen spawn point
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# Move the spectator behind the vehicle 
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform) 

# Add traffic to the simulation
for i in range(30): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))  
    
# Set the all vehicles in motion using the Traffic Manager
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True) 

Misc. Notes

In [ ]:
# Notes 

## Camera & sensor lists
#for bp in bp_lib.filter('camera'):
#    print(bp.id)
#for bp in bp_lib.filter('sensor'):
#    print(bp.id)

Spawn Camera

In [ ]:
# Spawn an RGB cammera with an offset from the vehicle center
camera_bp = bp_lib.find('sensor.camera.rgb') 
#camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.4)) 
camera_init_trans = carla.Transform(carla.Location(z=2.5, x=-3), carla.Rotation())
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# # Check camera set position 
# time.sleep(0.5) 
# spectator.set_transform(camera.get_transform())
# camera.destroy()

# # Start the camera saving data to disk
# camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))
# # Stop the camera when we've recorded enough data
# time.sleep(5) # 5Hz
# camera.stop()

In [ ]:
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
   
    #0605 
    # Send camera image data via ZeroMQ
    socket.send_pyobj({'type': 'camera', 'frame': image.frame, 'data': data_dict['image']})
    
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}

camera.listen(lambda image: camera_callback(image, camera_data))

Spawn Sensor

In [ ]:
# Color mapping for sensors
VIRIDIS = np.array(plt.colormaps.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(plt.colormaps.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:, :3]

def add_open3d_axis(vis):
    """Add a small 3D axis on Open3D Visualizer"""
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)


In [ ]:
def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity colors ready to be consumed by Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

#0605
    # Send LIDAR point cloud data via ZeroMQ
    socket.send_pyobj({'type': 'lidar', 'frame': point_cloud.frame, 'data': data[:, :-1]})


    # Isolate the intensity and compute a color for it
    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])]

    points = data[:, :-1]

    points[:, :1] = -points[:, :1]

    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

def radar_callback(data, point_list):
    radar_data = np.zeros((len(data), 4))

    for i, detection in enumerate(data):
        x = detection.depth * math.cos(detection.altitude) * math.cos(detection.azimuth)
        y = detection.depth * math.cos(detection.altitude) * math.sin(detection.azimuth)
        z = detection.depth * math.sin(detection.altitude)

        radar_data[i, :] = [x, y, z, detection.velocity]

    intensity = np.abs(radar_data[:, -1])
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, COOL_RANGE, COOL[:, 0]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 1]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 2])]

    points = radar_data[:, :-1]
    points[:, :1] = -points[:, :1]

    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)


In [ ]:
lidar_bp = bp_lib.find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '100.0')
lidar_bp.set_attribute('noise_stddev', '0.1')
lidar_bp.set_attribute('upper_fov', '15.0')
lidar_bp.set_attribute('lower_fov', '-25.0')
lidar_bp.set_attribute('channels', '64.0')
lidar_bp.set_attribute('rotation_frequency', '20.0')
lidar_bp.set_attribute('points_per_second', '500000')

lidar_init_trans = carla.Transform(carla.Location(z=2))
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)

radar_bp = bp_lib.find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '30.0')
radar_bp.set_attribute('vertical_fov', '30.0')
radar_bp.set_attribute('points_per_second', '10000')

radar_init_trans = carla.Transform(carla.Location(z=2))
radar = world.spawn_actor(radar_bp, radar_init_trans, attach_to=vehicle)

point_list = o3d.geometry.PointCloud()
radar_list = o3d.geometry.PointCloud()

lidar.listen(lambda data: lidar_callback(data, point_list))
radar.listen(lambda data: radar_callback(data, radar_list))

Computer Vision Windows

In [ ]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',
    width=960,
    height=540,
    left=480,
    top=270)

vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)

frame = 0
while True:
    if frame == 2:
        vis.add_geometry(point_list)
        vis.add_geometry(radar_list)
    vis.update_geometry(point_list)
    vis.update_geometry(radar_list)

    vis.poll_events()
    vis.update_renderer()
    # This can fix Open3D jittering issues:
    time.sleep(0.005)
    frame += 1

    cv2.imshow('RGB Camera', camera_data['image'])
    if cv2.waitKey(1) == ord('q'):
        break
        
cv2.destroyAllWindows()
radar.stop()
radar.destroy()
lidar.stop()
lidar.destroy()
camera.stop()
camera.destroy()
vis.destroy_window()

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

for actor in world.get_actors().filter('*sensor*'):
    actor.destroy()